In [12]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt 

def readPacketHist(path):
    files = os.listdir(path)[0:800]
    X_data = []
    for f in files:
        packets = readFromFile(path + f).split(" ")
        pkts = convert_stringArrays_to_floatArray(packets)
        hist = createHistFromPkst(pkts)
        if hist != "error":
            X_data.append(np.array(hist))
    return X_data


def readFromFile(path):
    with open(path, 'r') as content_file:
        content = content_file.read()
        return content


def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False


def convert_stringArrays_to_floatArray(array):
    intArray = []

    for k in array:
        if isfloat(k):
            intArray.append(float(k))
    return intArray


def buil_db(array, value):
    data_X = []
    data_Y = []
    n = len(array)
    for i in range(0, n):
        data_X.append(array[i])
        data_Y.append(value)

    return data_X, data_Y


def convert_stringArrays_to_intArray(array):
    intArray = []

    for k in array:
        if isfloat(k):
            intArray.append(int(k))
    return intArray


def createHistFromPkst(pkts):
    numberOfPkts = 1

    hist = [0] * 1515
    largerThanMTU = 0
    for i in range(0, len(pkts)):
        if pkts[i] not in [0, 40, 52]:
            if pkts[i] < len(hist):
                hist[int(pkts[i])] += 1
                numberOfPkts += 1
            else:
                largerThanMTU += 1

    for h in range(0, len(hist)):
        hist[h] /= float(numberOfPkts)
    if largerThanMTU / float((largerThanMTU + numberOfPkts)) < 0.05:
        return hist
    else:
        return "error"

def readFalseData():
    nTestDataFromEach = 200
    categories = ['httpMultipleTab/6','httpMultipleTab/7','httpMultipleTab/8','httpMultipleTab/5','httpMultipleTab/4'
                  ,'httpMultipleTab/3','httpMultipleTab/2','voip']

    x_train, y_train, x_test, y_test = [], [], [], []
    for n in range(len(categories)):
        dataset = readPacketHist(path='/home/fatemeh/Bitcoin/nov 13/hist/dataset/' + categories[n] + "/")
        print(categories[n], "  ReadFalse Data", len(dataset))
        data_x, data_y = buil_db(dataset[nTestDataFromEach:], 0)
        x_train.extend(data_x)
        y_train.extend(data_y)
        x_t, y_t = buil_db(dataset[0:nTestDataFromEach], 0)
        x_test.extend(x_t)
        y_test.extend(y_t)
    return (x_train, y_train), (x_test, y_test)


def readTrueData():
    nTestDataFromEach = 200
  
    categories = [ "noisyBitMultipleTab/6","noisyBitMultipleTab/7","noisyBitMultipleTab/8","noisyBitMultipleTab/5",
                 "noisyBitMultipleTab/4",
                 "noisyBitMultipleTab/3","noisyBitMultipleTab/2",'noisyBitcoinOverTor']
    x_train, y_train, x_test, y_test = [], [], [], []

    for n in range(len(categories)):
        dataset = readPacketHist(path='/home/fatemeh/Bitcoin/nov 13/hist/dataset/' + categories[n] + "/")
        print(categories[n], "Read TrueData", len(dataset))
        data_x, data_y = buil_db(dataset[nTestDataFromEach:], 1)
        x_train.extend(data_x)
        y_train.extend(data_y)
        x_t, y_t = buil_db(dataset[0:nTestDataFromEach], 1)
        x_test.extend(x_t)
        y_test.extend(y_t)

    return (x_train, y_train), (x_test, y_test)


def loadLargeDataset():
    (x_train, y_train), (x_test, y_test) = readFalseData()
    (x_train2, y_train2), (x_test2, y_test2) = readTrueData()
    x_train.extend(x_train2)
    y_train.extend(y_train2)
    x_test.extend(x_test2)
    y_test.extend(y_test2)
    x_test, y_test, x_train, y_train = np.array(x_test), np.array(y_test), np.array(x_train), np.array(y_train)
    return (x_train, y_train), (x_test, y_test)



'''
I should realize what to give as training data to my mode.
If I want to recognize bitcoin in presence of noise, should
I give noisyBitcoin as my training data? the problem here is when I increase the noise,
 results improves which makes
the approach fishy. I guess in these cases, I should increase the noise in my http too.
This makes it more sensible. When trying to analyze two tab
open noise + bitcoin, I should compare it with http which has 2 noise open.
'''


'\nI should realize what to give as training data to my mode.\nIf I want to recognize bitcoin in presence of noise, should\nI give noisyBitcoin as my training data? the problem here is when I increase the noise,\n results improves which makes\nthe approach fishy. I guess in these cases, I should increase the noise in my http too.\nThis makes it more sensible. When trying to analyze two tab\nopen noise + bitcoin, I should compare it with http which has 2 noise open.\n'

In [13]:

nTrainingData = 600
nTestData = 100
(x_train, y_train), (x_test, y_test) = loadLargeDataset() 

httpMultipleTab/6   ReadFalse Data 800
httpMultipleTab/7   ReadFalse Data 800
httpMultipleTab/8   ReadFalse Data 800
httpMultipleTab/5   ReadFalse Data 800
httpMultipleTab/4   ReadFalse Data 800
httpMultipleTab/3   ReadFalse Data 800
httpMultipleTab/2   ReadFalse Data 800
voip   ReadFalse Data 160
noisyBitMultipleTab/6 Read TrueData 295
noisyBitMultipleTab/7 Read TrueData 295
noisyBitMultipleTab/8 Read TrueData 295
noisyBitMultipleTab/5 Read TrueData 295
noisyBitMultipleTab/4 Read TrueData 295
noisyBitMultipleTab/3 Read TrueData 295
noisyBitMultipleTab/2 Read TrueData 295
noisyBitcoinOverTor Read TrueData 295


In [9]:

#what to do for a countermeasure:
#we have 800 http data, and rest of it is bitcoin traffic.

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # 'binary_crossentropy',
              metrics=['accuracy'])  # categorical_accuracy
# model.compile(loss='categorical_crossentropy',
# optimizer='sgd', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100)
model.evaluate(x_test, y_test)

Epoch 1/100
695/695 [==============================] - 1s 787us/step - loss: 0.5965 - acc: 0.8633
Epoch 2/100
695/695 [==============================] - 0s 215us/step - loss: 0.4576 - acc: 0.8633
Epoch 3/100
695/695 [==============================] - 0s 195us/step - loss: 0.3985 - acc: 0.8633
Epoch 4/100
695/695 [==============================] - 0s 198us/step - loss: 0.3909 - acc: 0.8633
Epoch 5/100
695/695 [==============================] - 0s 209us/step - loss: 0.3882 - acc: 0.8633
Epoch 6/100
695/695 [==============================] - 0s 194us/step - loss: 0.3868 - acc: 0.8633
Epoch 7/100
695/695 [==============================] - 0s 198us/step - loss: 0.3852 - acc: 0.8633
Epoch 8/100
695/695 [==============================] - 0s 199us/step - loss: 0.3830 - acc: 0.8633
Epoch 9/100
695/695 [==============================] - 0s 194us/step - loss: 0.3813 - acc: 0.8633
Epoch 10/100
695/695 [==============================] - 0s 197us/step - loss: 0.3791 - acc: 0.8633
Epoch 11/100
695/69

695/695 [==============================] - 0s 197us/step - loss: 0.0725 - acc: 0.9741
Epoch 84/100
695/695 [==============================] - 0s 195us/step - loss: 0.0717 - acc: 0.9741
Epoch 85/100
695/695 [==============================] - 0s 193us/step - loss: 0.0692 - acc: 0.9784
Epoch 86/100
695/695 [==============================] - 0s 194us/step - loss: 0.0669 - acc: 0.9784
Epoch 87/100
695/695 [==============================] - 0s 207us/step - loss: 0.0662 - acc: 0.9755
Epoch 88/100
695/695 [==============================] - 0s 214us/step - loss: 0.0652 - acc: 0.9784
Epoch 89/100
695/695 [==============================] - 0s 213us/step - loss: 0.0624 - acc: 0.9770
Epoch 90/100
695/695 [==============================] - 0s 206us/step - loss: 0.0621 - acc: 0.9799
Epoch 91/100
695/695 [==============================] - 0s 210us/step - loss: 0.0607 - acc: 0.9784
Epoch 92/100
695/695 [==============================] - 0s 220us/step - loss: 0.0587 - acc: 0.9784
Epoch 93/100
695/695 [=

[0.09591515600681305, 0.9725]